In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import math
import time
import mcfg #в файле лежат функции pg1,pg2 для подключения к postgresql

In [2]:
articles = 6657 #кол-во объявлений (ориентировочно, меняется ежеминутно)
num_on_page = 28 #отображается на одной странице
pages = math.ceil(articles/num_on_page) #страниц распарсить

def insert_to_pg(content):
    '''функция получает на вход текст страницы, забирает оттуда объявления и вносит в 3 колонки:
    айди объявления
    ссылка на объявление
    полный текст html элемента, который можно допарсить
    '''
    #CREATE TABLE team18.ads_list ( add_id text, add_link text, add_text text);
    soup = BeautifulSoup(content, "html.parser")
    ads = soup.findAll('article') #вытаскиваем по тегу все карточки
    for ad in ads:
        ad_link = ad.find('a', href=True)['href']
        full_ad = str(ad)
        ad_id = ad_link.split(r'/')[-2]


        sql = f"""insert into team18.ads_list (add_id,add_link,add_text) 
        VALUES ('{ad_id}',
        '{ad_link}',
        '{full_ad.replace("'",'"')}')

        """
        conn = mcfg.pg2()
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        conn.close()



for page in range(1,5):    #pages+1):
    
    link = f'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p={page}&region=1&room1=1&type=4'
    driver = webdriver.Chrome() 
    driver.get(link)
    time.sleep(10)
    content = driver.page_source
    driver.quit()
    insert_to_pg(content)
    
    print(f"{page}/{pages} done",end='\r')